# kmeans

In [1]:
from __future__ import print_function



import numpy as np

import tensorflow as tf

from tensorflow.contrib.factorization import KMeans



# Ignore all GPUs, tf k-means does not benefit from it.

import os

os.environ["CUDA_VISIBLE_DEVICES"] = ""



# Import MNIST data

from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("G:/Users/lenovo/AI-for-NLP/lesson10/tmp/data/", one_hot=True)

full_data_x = mnist.train.images



# Parameters

num_steps = 50 # Total steps to train

batch_size = 1024 # The number of samples per batch

k = 25 # The number of clusters

num_classes = 10 # The 10 digits

num_features = 784 # Each image is 28x28 pixels



# Input images

X = tf.placeholder(tf.float32, shape=[None, num_features])

# Labels (for assigning a label to a centroid and testing)

Y = tf.placeholder(tf.float32, shape=[None, num_classes])


#使用tensorflow集成的Kmeans方法
kmeans = KMeans(inputs=X, num_clusters=k, distance_metric='cosine',

                use_mini_batch=True)

# 构建kmeans图
training_graph = kmeans.training_graph()
# tensorflow版本不同所以启用不同参数 tensorflow1.4+多了一个参数
if len(training_graph) > 6: # Tensorflow 1.4+

    (all_scores, cluster_idx, scores, cluster_centers_initialized,

     cluster_centers_var, init_op, train_op) = training_graph

else:

    (all_scores, cluster_idx, scores, cluster_centers_initialized,

     init_op, train_op) = training_graph



cluster_idx = cluster_idx[0] # fix for cluster_idx being a tuple

avg_distance = tf.reduce_mean(scores)



# Initialize the variables (i.e. assign their default value)

init_vars = tf.global_variables_initializer()



# Start TensorFlow session

sess = tf.Session()



# Run the initializer

sess.run(init_vars, feed_dict={X: full_data_x})

sess.run(init_op, feed_dict={X: full_data_x})



# Training

for i in range(1, num_steps + 1):

    _, d, idx = sess.run([train_op, avg_distance, cluster_idx],

                         feed_dict={X: full_data_x})

    if i % 10 == 0 or i == 1:

        print("Step %i, Avg Distance: %f" % (i, d))



# Assign a label to each centroid

# Count total number of labels per centroid, using the label of each training

# sample to their closest centroid (given by 'idx')

counts = np.zeros(shape=(k, num_classes))

for i in range(len(idx)):

    counts[idx[i]] += mnist.train.labels[i]

# Assign the most frequent label to the centroid

labels_map = [np.argmax(c) for c in counts]

labels_map = tf.convert_to_tensor(labels_map)



# Evaluation ops

# Lookup: centroid_id -> label

cluster_label = tf.nn.embedding_lookup(labels_map, cluster_idx)

# Compute accuracy

correct_prediction = tf.equal(cluster_label, tf.cast(tf.argmax(Y, 1), tf.int32))
#tf.cast()函数的作用是执行 tensorflow 中张量数据类型转换
accuracy_op = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))



# Test Model

test_x, test_y = mnist.test.images, mnist.test.labels

print("Test Accuracy:", sess.run(accuracy_op, feed_dict={X: test_x, Y: test_y}))

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting G:/Users/lenovo/AI-for-NLP/lesson10/tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting G:/Users/lenovo/AI-for-NLP/lesson10/tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting G:/Users/lenovo/AI-for-NLP/lesson10/tmp/data/t10k-images-idx3-ubyte.gz
Extracting G:/Users/lenovo/AI-for-NLP/lesson10/tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
dim is deprecated, use axis instead
Step 1, Avg Distance: 0.341471
Step 10, Avg Distance: 0.221609
Step 20, Avg Distance: 0.220328
Step 30

In [2]:
full_data_x

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [3]:
full_data_x.shape[0]

55000

In [4]:
full_data_x.shape[1]

784